In [1]:
# Standard Library Imports (if any)

# Third-party Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from IPython.display import display, HTML
from adjustText import adjust_text
from sklearn.preprocessing import StandardScaler

# Local Module Imports
from utils import *
from data_handling import *
from plot import *
from data_handling.data_processor import *

In [2]:
major = COUNTRIES.MAJOR
offshore = COUNTRIES.OFFSHORE
sample = major + offshore
period = (2001, 2004)
dm = DataManager(
    raw_dir = "../data/raw",
    save_dir = "../data/clean"
)
ds_full = dm.get_dataset("ds")
fed_full = dm.get_dataset("fed")
# dm.clean_data()
# dm.save_data()
dm.filter_data(None, period)

In [3]:
# Access datasets
cpis = dm.get_dataset("cpis")
ds = dm.get_dataset("ds")
fed = dm.get_dataset("fed")
wb = dm.get_dataset("wb")
wfe = dm.get_dataset("wfe")
gdp = dm.get_dataset("gdp")

major = COUNTRIES.MAJOR
offshore = COUNTRIES.OFFSHORE
sample = major + offshore

In [9]:
weights_per_year = compute_weights(cpis, wb, major, offshore)
weights_dict = {}
for year in weights_per_year.columns:
    weights_dict[year] = weights_per_year[year].unstack("Country")
weights_avg = weights_per_year.mean(axis=1).unstack("Country")
index_returns = compute_index_excess_returns(ds, fed ,major)

means = pd.DataFrame(index=major)
vars = pd.DataFrame(index=major)
covs = pd.DataFrame(index=major)

for year, weights in weights_dict.items():
    returns = compute_excess_returns_statistics(index_returns, weights)
    means[year] = returns["mean_portfolio"]
    vars[year] = returns["var_portfolio"]  
    covs[year] = returns["cov_index_portfolio"]

returns = compute_excess_returns_statistics(index_returns, weights_avg)
means_avg = returns["mean_portfolio"]
vars_avg = returns["var_portfolio"]  
covs_avg = returns["cov_index_portfolio"]

In [11]:
print((means.var(axis=1) / (vars_avg / 12)).mean())

0.0006574869835458848


In [7]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects

# Access datasets
cpis = dm.get_dataset("cpis")
ds = dm.get_dataset("ds")
fed = dm.get_dataset("fed")
wb = dm.get_dataset("wb")
wfe = dm.get_dataset("wfe")
gdp = dm.get_dataset("gdp")
gdppc = dm.get_dataset("gdppc")

index_excess_returns = compute_index_excess_returns(ds_full, fed_full, major)
weights = compute_weights(cpis, wb, major, offshore).mean(axis=1).unstack(level="Country")
return_statistics = compute_excess_returns_statistics(index_excess_returns, weights)
domestic_investments = compute_domestic_investments(cpis, wb, major).droplevel(1)

y = (domestic_investments / gdp).stack() 
X_1 = gdppc.stack()
X_2 = return_statistics["mean_portfolio"]
X_3 = return_statistics["var_portfolio"]
# years = [2001, 2002, 2003, 2004]
# multi_index = pd.MultiIndex.from_product([X_2.index, years], names=['Country', 'Year'])
# X_2 = pd.Series(
#     X_2.values.repeat(len(years)),
#     index=multi_index,
#     name='Value'
# )
# X_3 = pd.Series(
#     X_3.values.repeat(len(years)),
#     index=multi_index,
#     name='Value'
# )
# X = pd.concat([X_1, X_2, X_3], axis=1, keys=["gdp/cap", "mean", "var"])
X = X_1
X = (X-X.mean())/X.std() # Normalize
X = sm.add_constant(X)

fe_model = PanelOLS(y, X, entity_effects=True, drop_absorbed=True).fit()
print(fe_model.summary)

fixed_effects = fe_model.estimated_effects.unstack(level="time").iloc[:,0]
X = pd.concat([X_2, X_3], axis=1, keys=["mean", "var"])
X = (X-X.mean())/X.std() # Normalize
X = sm.add_constant(X)
model_2 = sm.OLS(fixed_effects, X).fit()
print(model_2.summary())

                          PanelOLS Estimation Summary                           
Dep. Variable:                      0   R-squared:                        0.0179
Estimator:                   PanelOLS   R-squared (Between):             -0.0599
No. Observations:                 148   R-squared (Within):               0.0179
Date:                Thu, Apr 03 2025   R-squared (Overall):             -0.0537
Time:                        16:41:19   Log-likelihood                    54.807
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2.0045
Entities:                          37   P-value                           0.1597
Avg Obs:                       4.0000   Distribution:                   F(1,110)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             2.0045
                            

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from arch import arch_model

# Assume returns_df contains country-level stock returns
# and world_return is the global stock market return.

# Construct the independent variables
X = pd.DataFrame()
X["world_return"] = world_return
X["gdp_per_cap"] = gdp / wb.loc[major]
X["var"] = ds.var()  # Could use rolling window variance
X["gdp_growth"] = gdp.pct_change()  # GDP growth as proxy for expected future growth
X = X.dropna()

# Fit a GARCH(1,1)-M model
returns = ds.loc[X.index]  # Align data
garch_m = arch_model(returns, vol="Garch", p=1, q=1, mean="LS")
garch_m_fit = garch_m.fit(disp="off")

# Extract expected return (fitted value)
expected_returns = garch_m_fit.params["mu"] + X @ garch_m_fit.params.iloc[1:]


NameError: name 'world_return' is not defined

In [15]:
gdppc

,2001,2002,2003,2004
Country,,,,
AR,7141.475077,2569.699635,3320.477751,4242.020991
AT,24430.495983,26334.862215,32110.115966,36614.250653
BE,23015.071263,25006.191397,30655.209268,35429.407793
BR,3176.289357,2855.940189,3090.607010,3663.823188
CA,23822.096211,24256.019234,28301.864594,32145.541262
CL,4563.595299,4430.847412,4773.401775,6120.591521
CO,2472.990840,2428.929698,2311.942188,2818.571954
CZ,6669.074948,8101.260186,9852.456779,11782.554668
DK,30768.477547,33257.162259,40519.097491,46625.050098
